# Scraping of Fixable.com


In [2]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd

In [30]:
service_paths = ["","amazon-prime-video/","disney-plus/","hbo-max/","hulu/" ]
base_url = "https://flixable.com/"
paths = []

In [67]:
for service_path in service_paths:
    page = 1
    a = True
    while True:
        query = '?min-rating=0&min-year=1920&max-year=2022&order=date&page='
        response = requests.get(base_url + service_path + query + str(page))
        html = response.content
        soup = BeautifulSoup(html, 'html') 
        response_html = soup.find_all("div", {"id": "response"})[0]
        cards = response_html.findAll("div", {"class": "card-body"})
        if len(cards) > 1:
            page += 1
            print(page)
            for card in cards:
               paths.append(card.findAll('a', href=True)[0]['href'])
        else: 
            break

2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152


## Have all the paths, now get all the content, lets write the paths somewhere first

In [84]:
df=pd.DataFrame(paths, columns=['path']) 
df.to_csv("C:\\DCUGoogleDrive\\DataVisualisationScrapedData.csv")

The initial paths are grabbed, from there I need to do 1 main thing, loop through each row, attach it the path to the base url, query, then extract the data from the result
So hopefully the data is nicely formed


In [3]:
months = {
"January": "1",
"February": "2",
"March":"3",
"April":"4",
"May": "5",
"June":"6",
"July":"7",
"August":"8",
"September":"9",
"October":"10",
"November": "11",
"December": "12"
}

def remove_special_characters(title):
    new_text = ''
    for character in title:
        if character == ' ':
            new_text += '-'
        elif character.isalnum():
            new_text += character
    return new_text

In [60]:
def get_date(soup):
    x = soup.find('p', {'class':'mb-2'})
    if x is None:
        return '' 
    else :
        x = x.find_all('span')
        if len(x) < 2:
            return ''
        else :
            x = x[1].get_text().replace(',', '').split(' ')
            if len(x) < 3:
                return ''
            else :
                return x[2] + '-'+ months[x[0]] + '-' + x[1]

In [5]:
def get_title(soup):
    return soup.find('h1', {'class' : 'title'}).get_text()

In [6]:
def get_soup(path):
    response = requests.get(path)
    return BeautifulSoup(response.content, 'html') 

In [7]:
def get_source(path):
    arr = path.split('/')
    if arr[1] =='title':
        return 'netflix'
    else:
        return arr[1]
    

In [8]:
get_source('/amazon-prime-video/title/baby-ronnie-rhymes/')

'amazon-prime-video'

In [65]:
def get_sib_by_text(soup, el, text):
    x = soup.find("span", text=text)
    if x is None:
        return ''
    else:
        return x.next_sibling.get_text()

def get_director(soup):
    return get_sib_by_text(soup, "span", "Director:")

def get_genres(soup):
    return get_sib_by_text(soup, "span", "Genres:")

def get_cast(soup):
    return get_sib_by_text(soup, "span", "Cast:")

def get_production_country(soup):
    return get_sib_by_text(soup, "span", "Production Country:")

def get_rating(soup):
    x = soup.find("h6", {"class": "card-category"})   
    if x is None:
        return ''
    else:
        x = x.find("span", {"class": "fas fa-star rating mr-1"})
        if x is None:
            return '' 
        else :
            return x.next_sibling.get_text()

def get_duration(soup):
    x = soup.find("h6", {"class": "card-category"})
    if x is None:
        return ''
    else:
        x = x.find_all("span")
        if x is None:
            return ''
        else:
            if len(x) > 2:
                return x[2].get_text()
            else :
                return ''

def get_audience(soup):
    x = soup.find("h6", {"class": "card-category"}).find_all("span")[1]   
    if x is None:
        return ''
    else:
        return x.get_text()

def get_type(soup):
    if 'Season' in get_duration(soup):
        return 'tv-show'
    else:
        return 'movie'

def get_release_year(soup):
    x = soup.find('span', {'class' : 'mr-2'})
    if x is None:
        return ''
    else:
        return x.get_text()

def get_description(soup):
    x = soup.find('p', {'class' : 'card-description'})
    if x is None:
        return ''
    else:
        return x.get_text()


In [113]:
scrapedFrame = pd.read_csv("C:\\DCUGoogleDrive\\DataVisualisationScrapedFinal.csv")
df = pd.read_csv("C:\\DCUGoogleDrive\\DataVisualisationScrapedData.csv")


In [110]:
default_Titles = ['Full List of Movies and TV Shows on Amazon Prime Video | Flixable',
'Full List of Movies and TV Shows on Netflix | Flixable',
'Full List of Movies and Series on Disney+ | Flixable',
'Full List of Movies and Series on HBO Max | Flixable',
'Full List of Movies and TV Shows on Hulu | Flixable',]

In [114]:
#scrapedFrame = pd.DataFrame(columns = ['source', 'title', 'type', 'director', 'cast', 'production_countries', 'date_added','release_year', 'rating', 'duration', 'genres', 'description'])
scraped_rows = []
start_pos = len(scrapedFrame) + 1
for idx, row in df.iterrows():
    if idx > start_pos:
        soup = get_soup(base_url + row['path'])
        if soup.find('title').get_text() in default_Titles:
            print('skipping ' + row['path'])
        else:
            new_row = {
            'source':get_source(row['path'])
            ,'title':get_title(soup)
            ,'type':get_type(soup)
            ,'director':get_director(soup)
            ,'cast':get_cast(soup)
            ,'production_countries':get_production_country(soup)
            ,'date_added':get_date(soup)
            ,'release_year':get_release_year(soup)
            ,'rating': get_rating(soup)
            ,'audience' : get_audience(soup)
            ,'duration':get_duration(soup)
            ,'genres':get_genres(soup)
            ,'description':get_description(soup)}
            scraped_rows.append(new_row)
           # print(new_row)
        if idx % 100 == 0:
            print('Saving at row ' + str(idx))
            scrapedFrame = scrapedFrame.append(scraped_rows, ignore_index=True)
            scrapedFrame.to_csv("C:\\DCUGoogleDrive\\DataVisualisationScrapedFinal.csv")
            scraped_rows = []


scrapedFrame = scrapedFrame.append(scraped_rows, ignore_index=True)
scrapedFrame.to_csv("C:\\DCUGoogleDrive\\DataVisualisationScrapedFinal.csv")


Saving at row 20100
Saving at row 20200
Saving at row 20300
Saving at row 20400
Saving at row 20500
Saving at row 20600
Saving at row 20700
Saving at row 20800
Saving at row 20900
Saving at row 21000
Saving at row 21100
Saving at row 21200
Saving at row 21300
Saving at row 21400
Saving at row 21500
Saving at row 21600
Saving at row 21700
Saving at row 21800
Saving at row 21900
Saving at row 22000
Saving at row 22100
Saving at row 22200
Saving at row 22300
Saving at row 22400
Saving at row 22500
Saving at row 22600
Saving at row 22700
Saving at row 22800
Saving at row 22900
Saving at row 23000
Saving at row 23100
Saving at row 23200
Saving at row 23300
Saving at row 23400
Saving at row 23500
Saving at row 23600
Saving at row 23700
Saving at row 23800
Saving at row 23900
Saving at row 24000
Saving at row 24100
Saving at row 24200
skipping /hulu/title/the-cup/
Saving at row 24300
Saving at row 24400
Saving at row 24500
Saving at row 24600
Saving at row 24700
Saving at row 24800
Saving at 

In [79]:
soup = get_soup('https://flixable.com/hulu/')
soup.find('title').get_text()

'Full List of Movies and TV Shows on Hulu | Flixable'

In [106]:
scrapedFrame.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   source                50 non-null     object
 1   title                 50 non-null     object
 2   type                  50 non-null     object
 3   director              50 non-null     object
 4   cast                  50 non-null     object
 5   production_countries  50 non-null     object
 6   date_added            50 non-null     object
 7   release_year          50 non-null     object
 8   rating                50 non-null     object
 9   audience              50 non-null     object
 10  duration              50 non-null     object
 11  genres                50 non-null     object
 12  description           50 non-null     object
dtypes: object(13)
memory usage: 5.2+ KB
